In [3]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
train_path = "train/"
test_path = "test/"

In [5]:
train_batch = ImageDataGenerator(rescale=1./255).flow_from_directory(train_path,target_size=(224,224),batch_size=10,classes=['cats','dogs'])
test_batch = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,target_size=(224,224),batch_size=5,classes=['cats','dogs'])

Found 100 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [6]:
vgg_model = keras.applications.VGG16()
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [7]:
type(vgg_model)  # we got Model type model which is keras functional api we have to make it sequetial

keras.engine.training.Model

Removing last layer from model because vgg16 classify 1000 classes but we want only 2 classes


In [8]:
vgg_model.layers.pop()

In [9]:
model = Sequential() #making sequential model
for layer in vgg_model.layers: #copying all the VGG16 model layer to our new Sequential model
    model.add(layer)

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [11]:
for layers in vgg_model.layers: # we dont want to touch pre-trained weight
    layers.trainable = False

In [12]:
model.add(Dense(2,activation='softmax')) #adding last layer which classify our custom category

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [14]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [47]:
model.fit_generator(train_batch,epochs=5,verbose=1)

Epoch 1/5
10/10 [==============================] - 53s 5s/step - loss: 0.3037 - acc: 0.8800
Epoch 2/5
10/10 [==============================] - 55s 5s/step - loss: 0.2686 - acc: 0.9100
Epoch 3/5
10/10 [==============================] - 55s 5s/step - loss: 0.2601 - acc: 0.9100
Epoch 4/5
10/10 [==============================] - 54s 5s/step - loss: 0.2401 - acc: 0.9400
Epoch 5/5
10/10 [==============================] - 54s 5s/step - loss: 0.2344 - acc: 0.9200


In [48]:
accuracy = model.evaluate_generator(test_batch,verbose=1)

6/6 [==============================] - 15s 3s/step


In [49]:
accuracy[1]*100

76.66666805744171

### Prediction on test images

In [50]:
preds = model.predict_generator(test_batch,verbose=1)
preds

6/6 [==============================] - 17s 3s/step


array([[0.7100395 , 0.2899605 ],
       [0.8126494 , 0.18735051],
       [0.9579981 , 0.04200183],
       [0.70527714, 0.29472286],
       [0.9664614 , 0.03353859],
       [0.5333554 , 0.46664453],
       [0.96356535, 0.03643469],
       [0.12402468, 0.8759754 ],
       [0.87320834, 0.12679167],
       [0.68562376, 0.31437626],
       [0.19507074, 0.80492926],
       [0.23404917, 0.76595086],
       [0.01542814, 0.9845718 ],
       [0.8162008 , 0.18379918],
       [0.9352192 , 0.06478074],
       [0.97171783, 0.02828218],
       [0.9693498 , 0.03065016],
       [0.3435213 , 0.6564787 ],
       [0.24702229, 0.75297767],
       [0.82131153, 0.17868847],
       [0.91234815, 0.08765181],
       [0.06692959, 0.9330704 ],
       [0.32751855, 0.6724814 ],
       [0.59025705, 0.40974292],
       [0.0023428 , 0.9976572 ],
       [0.52239025, 0.47760984],
       [0.52849084, 0.47150913],
       [0.8710931 , 0.12890689],
       [0.98369294, 0.016307  ],
       [0.00604732, 0.9939527 ]], dtype=flo

In [46]:
true_label = []
test_batch.batch_index = 0
for batch in range(6):
    imgs,labels = next(test_batch)
    true_label.append(labels)
    #print(test_batch.batch_index)
true_label    

[array([[0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]], dtype=float32), array([[1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.]], dtype=float32), array([[1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.]], dtype=float32), array([[1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.]], dtype=float32), array([[0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.]], dtype=float32), array([[0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.]], dtype=float32)]

### prediction on single image

In [51]:
from keras.preprocessing import image
img = image.load_img("tabby-cat-names.jpg",target_size=(224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
x = x/255
pred = model.predict(x)
pred   # Index 0 got .9 probability and index 0 belong to cat

array([[0.6506391, 0.3493609]], dtype=float32)

In [52]:
train_batch.class_indices

{'cats': 0, 'dogs': 1}

In [53]:
#classwise prediction
model.predict_classes(x) # 0 belong to cat

array([0])

In [54]:
#saving model for future fun
model.save("my_first_model.h5")